In [1]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl,numpy as np,matplotlib.pyplot as plt
from pathlib import Path
from torch import tensor,nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from fastcore.test import test_close

torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [2]:
from datasets import load_dataset,load_dataset_builder
import logging

In [3]:
logging.disable(logging.WARNING)

## Hugging Face Datasets

In [4]:
name = "fashion_mnist"
ds_builder = load_dataset_builder(name)
print(ds_builder.info.description)

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of
60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image,
associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in
replacement for the original MNIST dataset for benchmarking machine learning algorithms.
It shares the same image size and structure of training and testing splits.



In [5]:
ds_builder.info.features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(num_classes=10, names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)}

In [6]:
ds_builder.info.splits

{'train': SplitInfo(name='train', num_bytes=31296607, num_examples=60000, dataset_name='fashion_mnist'),
 'test': SplitInfo(name='test', num_bytes=5233810, num_examples=10000, dataset_name='fashion_mnist')}

In [7]:
ds = load_dataset("fashion_mnist")
ds

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [8]:
train,test = ds['train'],ds['test']

In [9]:
train[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 9}

In [10]:
x,y = 'image','label'

In [11]:
img = train[0][x]
img

In [12]:
train[:10][y]

[9, 0, 0, 3, 0, 2, 7, 2, 5, 5]

In [14]:
featy = train.features[y]
featy.int2str(train[:5][y])

['Ankle boot',
 'T - shirt / top',
 'T - shirt / top',
 'Dress',
 'T - shirt / top']

In [15]:
from torch.utils.data import DataLoader

In [16]:
def collate_fn(b):
    return {x:torch.stack([TF.to_tensor(o[x]) for o in b]),
            y:tensor([o[y] for o in b])}

In [18]:
dl = DataLoader(train, collate_fn=collate_fn, batch_size=4)
b = next(iter(dl))
b[x].shape,b[y]

(torch.Size([4, 1, 28, 28]), tensor([9, 0, 0, 3]))

In [20]:
def transforms(b):
    b[x] = [TF.to_tensor(o) for o in b[x]]
    return b

In [23]:
tds = train.with_transform(transforms)
dl = DataLoader(tds, batch_size=4)
b = next(iter(dl))
b[x].shape,b[y]

(torch.Size([4, 1, 28, 28]), tensor([9, 0, 0, 3]))

In [28]:
def inplace(f):
    def _f(b):
        f(b)
        return b
    return _f

In [32]:
@inplace
def transformi(b): b[x] = [TF.to_tensor(o) for o in b[x]]

In [33]:
tds = train.with_transform(transformi)
dl = DataLoader(tds, batch_size=4)
b = next(iter(dl))
b[x].shape,b[y]

(torch.Size([4, 1, 28, 28]), tensor([9, 0, 0, 3]))